## Data Cleaning

This notebook has the purpose of: 
- Cleaning and to combine all the reviews into one table.
- Create a wine table, extracted from the the reviews. one wine can be reviewed many times and we need a wine list for each vineyard.

In [1]:
import os
import re
import numpy as np
import pandas as pd
from collections import Counter

raw_spectator_file = "combined_spectator_reviews.csv"
raw_enthusiast_file = "raw_wine_enthusiast_reviews.csv"


In [2]:
spectator_df = pd.read_csv(raw_spectator_file)
spectator_df

,vintage,score,release_price,bottle_name,winery,review_text,Magazine,review_link,region,source,varietal,wine_id,Region
0,2011,91,$40,Cabernet Franc Rogue Valley,Griffin Creek,"Soft, ripe, generous and precise, this open-te...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Franc,379875.0,NaN
1,2011,86,$40,Arsenal Rogue Valley,Quady North,"A snappy red, if modest in scale, with blackbe...","Magazine Issue Feb 02, 2017",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Franc,429948.0,NaN
2,2011,90,$28,Cabernet Sauvignon Rogue Valley,Del Rio,"A vibrant, medium-weight red, with ripe plum a...","Magazine Issue Feb 28, 2014",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Sauvignon & Blends,357733.0,NaN
3,2011,86,$40,Cabernet Sauvignon Rogue Valley,Griffin Creek,"Lean, spicy and firmly tannic, offering billow...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Southern Oregon', 'Rogue Valley']",Wine Spectator,Cabernet Sauvignon & Blends,379943.0,NaN
4,2011,94,$125,Chardonnay Dundee Hills Récolte Grand Cru,Domaine Serene,"Tangy, with vibrant acidity and expressive, ri...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Wine Spectator,Chardonnay,380170.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,2015,90,$45,Chardonnay Dundee Hills,Knudsen,"Crisp and harmonious, with lemon blossom and a...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Dundee Hills']",Wine Spectator,Chardonnay,434673.0,NaN
1234,2015,90,$40,Chardonnay Willamette Valley Reserve,Ponzi,"Graceful and polished, with expressive pear, s...","Magazine Issue Jul 31, 2018",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator,Chardonnay,454785.0,NaN
1235,2015,90,$45,Chardonnay Yamhill-Carlton District Gran Morai...,Raptor Ridge,"Light and polished, showing pretty pear, pinea...",Magazine Issue Web Only - 2017,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,427274.0,NaN
1236,2015,90,$60,Chardonnay Yamhill-Carlton District Marsh Esta...,Roco,"Floral and harmonious, with delicate tropical ...",Magazine Issue Web Only - 2018,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator,Chardonnay,447448.0,NaN


In [6]:
enthusiast_df = pd.read_csv(raw_enthusiast_file)
enthusiast_df

,Unnamed: 0,price,rating,bottle_name,region,review_link,source,Price,Designation,Variety,Appellation,Winery,Alcohol,Bottle Size,Category,Date Published,User Avg Rating,Importer
0,0,$13,88,The Great Oregon Wine Company 2021 Rascal Pino...,Oregon Other,https://www.winemag.com/buying-guide/the-great...,Wine Enthusiast,"['$13', 'Buy Now']",['Rascal'],['Pinot Noir'],"['Oregon', ' Oregon Other', ' Oregon', ' US']",['The Great Oregon Wine Company'],['13.3%'],['750 ml'],['Red'],['10/1/2022'],['Not rated yet [Add Your Review]'],NaN
1,1,$19,86,Wine by Joe 2019 Pinot Noir (Oregon),Oregon Other,https://www.winemag.com/buying-guide/wine-by-j...,Wine Enthusiast,"['$19', 'Buy Now']",NaN,['Pinot Noir'],"['Oregon', ' Oregon Other', ' Oregon', ' US']",['Wine by Joe'],['13.5%'],['750 ml'],['Red'],['10/1/2022'],['Not rated yet [Add Your Review]'],NaN
2,2,$15,88,Duck Pond Cellars 2021 Oregon Wine Co. Chardon...,Oregon Other,https://www.winemag.com/buying-guide/duck-pond...,Wine Enthusiast,"['$15', 'Buy Now']",['Oregon Wine Co.'],['Chardonnay'],"['Oregon', ' Oregon Other', ' Oregon', ' US']",['Duck Pond Cellars'],['13.2%'],['750 ml'],['White'],['9/1/2022'],['Not rated yet [Add Your Review]'],NaN
3,3,$75,92,Domaine Drouhin 2019 Laurene Pinot Noir (Dunde...,Willamette Valley,https://www.winemag.com/buying-guide/domaine-d...,Wine Enthusiast,"['$75', 'Buy Now']",['Laurene'],['Pinot Noir'],"['Dundee Hills', ' Willamette Valley', ' Orego...",['Domaine Drouhin'],['13.9%'],['750 ml'],['Red'],['7/1/2022'],['Not rated yet [Add Your Review]'],NaN
4,4,$40,90,Portlandia 2019 Momtazi Vineyard Pinot Noir (W...,Willamette Valley,https://www.winemag.com/buying-guide/portlandi...,Wine Enthusiast,"['$40', 'Buy Now']",['Momtazi Vineyard'],['Pinot Noir'],"['Willamette Valley', ' Willamette Valley', ' ...",['Portlandia'],['13.5%'],['750 ml'],['Red'],['7/1/2022'],['Not rated yet [Add Your Review]'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,736,$18,90,Ponzi 1998 Arneis (Willamette Valley),Willamette Valley,https://www.winemag.com/buying-guide/ponzi-199...,Wine Enthusiast,"['$18', 'Buy Now']",NaN,"['Arneis', ' Italian White']","['Willamette Valley', ' Willamette Valley', ' ...",['Ponzi'],['N/A'],['500 ml'],['White'],['4/1/2000'],['Not rated yet [Add Your Review]'],NaN
737,737,$12,86,Tyee 1996 Pinot Blanc (Willamette Valley),Willamette Valley,https://www.winemag.com/buying-guide/tyee-1996...,Wine Enthusiast,"['$12', 'Buy Now']",NaN,['Pinot Blanc'],"['Willamette Valley', ' Willamette Valley', ' ...",['Tyee'],['13%'],['750 ml'],['White'],['12/31/1999'],['Not rated yet [Add Your Review]'],NaN
738,738,$35,92,Cameron 1997 Clos Electrique Chardonnay (Willa...,Willamette Valley,https://www.winemag.com/buying-guide/cameron-1...,Wine Enthusiast,"['$35', 'Buy Now']",['Clos Electrique'],['Chardonnay'],"['Willamette Valley', ' Willamette Valley', ' ...",['Cameron'],['12.7%'],['750 ml'],['White'],['11/1/1999'],['Not rated yet [Add Your Review]'],NaN
739,739,$15,90,Torii Mor 1998 Pinot Gris (Yamhill County),Willamette Valley,https://www.winemag.com/buying-guide/torii-mor...,Wine Enthusiast,"['$15', 'Buy Now']",NaN,['Pinot Gris'],"['Yamhill County', ' Willamette Valley', ' Ore...",['Torii Mor'],['N/A'],['750 ml'],['White'],['11/1/1999'],['Not rated yet [Add Your Review]'],NaN


In [3]:
spectator_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238 entries, 0 to 1237
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   vintage        1238 non-null   int64  
 1   score          1238 non-null   int64  
 2   release_price  1238 non-null   object 
 3   bottle_name    1238 non-null   object 
 4   winery         1238 non-null   object 
 5   review_text    1238 non-null   object 
 6   Magazine       1238 non-null   object 
 7   review_link    1238 non-null   object 
 8   region         865 non-null    object 
 9   source         1238 non-null   object 
 10  varietal       865 non-null    object 
 11  wine_id        865 non-null    float64
 12  Region         373 non-null    object 
dtypes: float64(1), int64(2), object(10)
memory usage: 125.9+ KB


In [4]:
spectator_df["winery"].value_counts()

Domaine Serene       57
Bergström            56
Evening Land         45
Beaux Frères         41
Alexana              38
                     ..
Coelho                1
Ancient Cellars       1
Anam Cara Cellars     1
Et Fille              1
Authentique           1
Name: winery, Length: 196, dtype: int64

In [7]:
enthusiast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741 entries, 0 to 740
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       741 non-null    int64 
 1   price            739 non-null    object
 2   rating           741 non-null    int64 
 3   bottle_name      741 non-null    object
 4   region           741 non-null    object
 5   review_link      741 non-null    object
 6   source           741 non-null    object
 7   Price            741 non-null    object
 8   Designation      427 non-null    object
 9   Variety          741 non-null    object
 10  Appellation      741 non-null    object
 11  Winery           741 non-null    object
 12  Alcohol          741 non-null    object
 13  Bottle Size      741 non-null    object
 14  Category         741 non-null    object
 15  Date Published   741 non-null    object
 16  User Avg Rating  741 non-null    object
 17  Importer         4 non-null      ob

Starting with the Wine Enthusiast reviews. They seem to be the dirtiest so far

In [24]:
## First task is to get rid of the list appearance of the string formatted data
cleaning_df = enthusiast_df.drop(['Price', "Unnamed: 0", "Importer"], axis=1)
# designation = cleaning_df['Designation']
alcohol = cleaning_df['Alcohol']
alcohol

0      ['13.3%']
1      ['13.5%']
2      ['13.2%']
3      ['13.9%']
4      ['13.5%']
         ...    
736      ['N/A']
737      ['13%']
738    ['12.7%']
739      ['N/A']
740    ['13.2%']
Name: Alcohol, Length: 741, dtype: object

In [25]:
for i in alcohol.index:
    save = re.search(r"\d+\.*\d*%", alcohol[i])
    if save != None:
        alcohol[i] = save.group()
    else:
        alcohol[i] = None
alcohol

C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\4171088931.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alcohol[i] = save.group()
C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\4171088931.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alcohol[i] = None


0      13.3%
1      13.5%
2      13.2%
3      13.9%
4      13.5%
       ...  
736     None
737      13%
738    12.7%
739     None
740    13.2%
Name: Alcohol, Length: 741, dtype: object

In [26]:
cleaning_df['Alcohol'] = alcohol
size = cleaning_df['Bottle Size']
size

0      ['750 ml']
1      ['750 ml']
2      ['750 ml']
3      ['750 ml']
4      ['750 ml']
          ...    
736    ['500 ml']
737    ['750 ml']
738    ['750 ml']
739    ['750 ml']
740    ['750 ml']
Name: Bottle Size, Length: 741, dtype: object

In [55]:
for i in size.index:
    keep = re.search(r"(\d+\s*ml)", size[i])
    if keep != None:
        pass
        size[i] = keep.group()
    else:
        keep = re.search(r"(\d+\s*ML)", size[i])
        if keep != None:
            pass
            size[i] = keep.group()
        else: 
            keep = re.search(r"(\d+\.\d*\s*L)", size[i])
            size[i] = keep.group()

size

C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\2907954254.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  size[i] = keep.group()
C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\2907954254.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  size[i] = keep.group()
C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\2907954254.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  size[i] = keep.group()


0      750 ml
1      750 ml
2      750 ml
3      750 ml
4      750 ml
        ...  
736    500 ml
737    750 ml
738    750 ml
739    750 ml
740    750 ml
Name: Bottle Size, Length: 741, dtype: object

In [57]:
cleaning_df['Bottle Size'] = size
winery = cleaning_df['Winery']
winery

0      ['The Great Oregon Wine Company']
1                        ['Wine by Joe']
2                  ['Duck Pond Cellars']
3                    ['Domaine Drouhin']
4                         ['Portlandia']
                     ...                
736                            ['Ponzi']
737                             ['Tyee']
738                          ['Cameron']
739                        ['Torii Mor']
740               ['WillaKenzie Estate']
Name: Winery, Length: 741, dtype: object

In [60]:
for i in winery.index:
    winery[i] = winery[i][2:-2]
    
winery    

C:\Users\Nick\AppData\Local\Temp\ipykernel_16248\98839780.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  winery[i] = winery[i][2:-2]


0      The Great Oregon Wine Company
1                        Wine by Joe
2                  Duck Pond Cellars
3                    Domaine Drouhin
4                         Portlandia
                   ...              
736                            Ponzi
737                             Tyee
738                          Cameron
739                        Torii Mor
740               WillaKenzie Estate
Name: Winery, Length: 741, dtype: object

In [61]:
cleaning_df["Winery"] = winery
variety = cleaning_df["Variety"]
variety

0                    ['Pinot Noir']
1                    ['Pinot Noir']
2                    ['Chardonnay']
3                    ['Pinot Noir']
4                    ['Pinot Noir']
                   ...             
736    ['Arneis', ' Italian White']
737                 ['Pinot Blanc']
738                  ['Chardonnay']
739                  ['Pinot Gris']
740                ['Pinot Grigio']
Name: Variety, Length: 741, dtype: object

In [104]:
category = ['Red','White','Sparkling','Rose','Dessert']
for i in variety.index:
    grape = variety[i][2:-2].split("', '")
#     if len(grape) >1:
    if (str(grape) == "['Sparkling Blend', ' Sparkling']"):
#             print(i,grape)
#             grape = "Blend"
            pass
    elif 'Rosé' in grape:
##    'Rosé' is a Wine category not a grape. After viewing the reviews by following the review_links
##    the following changes will be made (index, new designation) :  (16, Blend), (80, Pinot Noir), (175, Pinot Noir),
##    (232, Blend), (239, Blend), (240, Blend), (241, Blend), (253, Grenache), (309, Pinot Noir), (310, Blend), (486, Blend)
##    (602, Sangiovese)

## 486 grenache Pinot Noir
## 232, grenache-Syrah   
## 310, NV
        print(i,grape, '\n', cleaning_df['review_link'][i])
#         pass
    else: 
        pass
#         print(grape)
    



16 ['Rosé'] 
 https://www.winemag.com/buying-guide/j-christopher-2020-cristo-irresisto-oregon-pink-wine-rose-oregon/
80 ['Rosé'] 
 https://www.winemag.com/buying-guide/rex-hill-2019-pinot-noir-rose-willamette-valley/
175 ['Rosé'] 
 https://www.winemag.com/buying-guide/toil-oregon-2018-pinot-noir-rose-willamette-valley/
232 ['Rosé'] 
 https://www.winemag.com/buying-guide/j-christopher-2017-cristo-irresisto-pink-wine-rose-oregon/
239 ['Rosé'] 
 https://www.winemag.com/buying-guide/the-great-oregon-wine-company-2017-rose-city-rose-willamette-valley/
240 ['Rosé'] 
 https://www.winemag.com/buying-guide/conveyance-2017-rose-willamette-valley/
241 ['Rosé'] 
 https://www.winemag.com/buying-guide/domaine-loubejac-2017-rose-willamette-valley/
242 ['Rosé'] 
 https://www.winemag.com/buying-guide/project-m-2017-acclara-rose-willamette-valley/
247 ['Rosé'] 
 https://www.winemag.com/buying-guide/samuel-robert-2017-vintners-reserve-pinot-noir-rose-willamette-valley/
253 ['Rosé'] 
 https://www.winemag.

In [105]:
cleaning_df.loc[310,]

price                                                            $13
rating                                                            85
bottle_name           The Great Oregon Wine Company NV Rosé (Oregon)
region                                                  Oregon Other
review_link        https://www.winemag.com/buying-guide/the-great...
source                                               Wine Enthusiast
Designation                                                      NaN
Variety                                                     ['Rosé']
Appellation            ['Oregon', ' Oregon Other', ' Oregon', ' US']
Winery                                 The Great Oregon Wine Company
Alcohol                                                          13%
Bottle Size                                                   187 ml
Category                                                    ['Rose']
Date Published                                          ['8/1/2017']
User Avg Rating                  [

In [107]:
cleaning_df.keys()

Index(['price', 'rating', 'bottle_name', 'region', 'review_link', 'source',
       'Designation', 'Variety', 'Appellation', 'Winery', 'Alcohol',
       'Bottle Size', 'Category', 'Date Published', 'User Avg Rating'],
      dtype='object')

In [72]:
reviews = spectator_df["review_text"]
reviews[355]

'Offering verve and persistence, this Pinot is taut at first but slowly blooms into dynamic blueberry and black cherry flavors that are accented by forest floor and spice tea. Best from 2023 through 2031. 334 cases made.'

In [80]:
spam = Counter()
for i, review in enumerate(reviews):
    if " Drink now through" in review:
        cut_line = review.index(" Drink now through")
        review = review[:cut_line]
#         print(len(review)) 
    elif " Drink now." in review:
        cut_line = review.index(" Drink now.")
        review = review[:cut_line]
#         print(len(review)) [133,151,190]
    else:
        try:
            check = review.index(" Best from ")
        except:
            print(review)#[:check])
    
#     spam.update(x.split())

spam    

Impressively built and expressive, with vibrant rose petal, black cherry and orange peel flavors that take on richness and depth toward polished tannins. Best after 2025. 610 cases made.
Impressively built and expressive, with vibrant rose petal, black cherry and orange peel flavors that take on richness and depth toward polished tannins. Best after 2025. 610 cases made.
Impressively built and expressive, with vibrant rose petal, black cherry and orange peel flavors that take on richness and depth toward polished tannins. Best after 2025. 610 cases made.
Impressively built and expressive, with vibrant rose petal, black cherry and orange peel flavors that take on richness and depth toward polished tannins. Best after 2025. 610 cases made.
Impressively built and expressive, with vibrant rose petal, black cherry and orange peel flavors that take on richness and depth toward polished tannins. Best after 2025. 610 cases made.
Impressively built and expressive, with vibrant rose petal, black

Counter()

In [89]:
drop_dup = spectator_df.drop_duplicates()
drop_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 373 entries, 0 to 4472
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   vintage        373 non-null    int64 
 1   score          373 non-null    int64 
 2   release_price  373 non-null    object
 3   bottle_name    373 non-null    object
 4   winery         373 non-null    object
 5   review_text    373 non-null    object
 6   Magazine       373 non-null    object
 7   review_link    373 non-null    object
 8   Region         373 non-null    object
 9   source         373 non-null    object
dtypes: int64(2), object(8)
memory usage: 32.1+ KB


In [98]:
spectator_df[spectator_df["bottle_name"] == "Pinot Noir Willamette Valley"]#["review_link"][8]

,vintage,score,release_price,bottle_name,winery,review_text,Magazine,review_link,Region,source
8,2019,95,$55,Pinot Noir Willamette Valley,Nicolas-Jay,"A lovely wine, supple and harmonious, with mul...",Magazine Issue Web Only - 2022,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator
17,2019,94,$65,Pinot Noir Willamette Valley,Beaux Frères,"Vibrant and detailed, with impeccably structur...","Magazine Issue Oct 31, 2021",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator
47,2019,94,$35,Pinot Noir Willamette Valley,Résonance,"Vibrant and impeccably structured, with floral...","Magazine Issue Mar 31, 2022",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator
58,2019,95,$55,Pinot Noir Willamette Valley,Nicolas-Jay,"A lovely wine, supple and harmonious, with mul...",Magazine Issue Web Only - 2022,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator
67,2019,94,$65,Pinot Noir Willamette Valley,Beaux Frères,"Vibrant and detailed, with impeccably structur...","Magazine Issue Oct 31, 2021",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator
...,...,...,...,...,...,...,...,...,...,...
4363,2012,95,$50,Pinot Noir Willamette Valley,Bomb,"Supple, expressive, multilayered and harmoniou...","Magazine Issue Jul 31, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator
4380,2012,94,$50,Pinot Noir Willamette Valley,Cornerstone,"Supple, ripe and expressive, unfurling ripe bl...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator
4412,2012,95,$40,Pinot Noir Willamette Valley,Big Table Farm,"Vibrant and intense, pulsing with mint-edged r...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator
4413,2012,95,$50,Pinot Noir Willamette Valley,Bomb,"Supple, expressive, multilayered and harmoniou...","Magazine Issue Jul 31, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator


In [100]:
spectator_df[spectator_df.duplicated()==True]

,vintage,score,release_price,bottle_name,winery,review_text,Magazine,review_link,Region,source
50,2019,96,$65,Pinot Noir Yamhill-Carlton Résonance Vineyard,Résonance,"Impressive and compelling, this Pinot epitomiz...","Magazine Issue Jul 31, 2022",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Yamhill-Carlt...",Wine Spectator
51,2019,95,$80,Pinot Noir Chehalem Mountains Silice,Bergström,"Polished, rich and luscious, offering vibrant ...",Magazine Issue Web Only - 2021,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Chehalem Moun...",Wine Spectator
52,2019,95,$80,Pinot Noir Ribbon Ridge Le Pré Du Col Vineyard,Bergström,"A lovely red, refined and generous, with vibra...",Magazine Issue Web Only - 2021,https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Ribbon Ridge']",Wine Spectator
53,2019,95,$35,Pinot Noir Eola-Amity Hills,Drouhin Oregon Roserock,This version is defined by a beam of vibrant a...,"Magazine Issue Feb 28, 2022",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Eola-Amity Hi...",Wine Spectator
54,2019,95,$100,Chardonnay Eola-Amity Hills Summum Seven Sprin...,Evening Land,"Shows dimension and presence, with elegantly m...","Magazine Issue Aug 31, 2022",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Eola-Amity Hi...",Wine Spectator
...,...,...,...,...,...,...,...,...,...,...
4445,2012,94,$100,Pinot Noir Willamette Valley Pas De Nom,Penner-Ash,"Light on its feet, offering rich black cherry ...","Magazine Issue Oct 15, 2014",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator
4446,2012,94,$60,Chardonnay Chehalem Mountains Aurora,Ponzi,"Taut, focused and expressive, aligning vibrant...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'Chehalem Moun...",Wine Spectator
4447,2012,94,$45,Pinot Noir McMinnville Meredith Mitchell Vineyard,Raptor Ridge,"Fresh and vital, with juicy blackberry and boy...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley', 'McMinnville']",Wine Spectator
4448,2012,94,$50,Pinot Noir Willamette Valley The Stalker,Roco,"Sleek and racy, with vibrant notes of black ch...","Magazine Issue Feb 28, 2015",https://www.winespectator.com/wine/wine-detail...,"['Oregon', 'Willamette Valley']",Wine Spectator


In [105]:
duplicate_test = set()

for i in spectator_df.index:
    spam = str()
    for col in spectator_df.columns:
        spam += str(spectator_df[col][i])
    
    duplicate_test.add(spam)
    
len(duplicate_test)    

373

In [107]:
for name in spectator_df["bottle_name"]:
    if "pinot noir" not in name.lower():
        if "chardonnay" not in name.lower():
            print(name)

Brut Rosé Willamette Valley
Brut Rosé Willamette Valley
Brut Rosé Willamette Valley
Brut Rosé Willamette Valley
Brut Rosé Willamette Valley
Brut Rosé Willamette Valley
Brut Rosé Willamette Valley
Brut Rosé Willamette Valley
Brut Rosé Willamette Valley
Brut Rosé Willamette Valley
Pinot Gris Willamette Valley
Pinot Gris Willamette Valley
Pinot Gris Willamette Valley
Rosé Oregon
Grenache Rogue Valley Rosé
Rosé Oregon
Riesling Oregon
Pinot Gris Willamette Valley
Pinot Gris Oregon


In [110]:
enthusiest_df = pd.read_csv(raw_enthusiest_file)

NameError: name 'raw_enthusiest_file' is not defined